In [33]:
!pip install natsort
import os
import shutil
from natsort import natsorted
import re

import json
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import glob

sns.set(style='darkgrid')

# M3IL(ours 70.4%)

In [34]:

data = pd.DataFrame(index=[i for i in range(25000, 400001, 25000)])
print(data)

Empty DataFrame
Columns: []
Index: [25000, 50000, 75000, 100000, 125000, 150000, 175000, 200000, 225000, 250000, 275000, 300000, 325000, 350000, 375000, 400000]


In [35]:
def get_score_from_json(file):
    try:
        with open(file + '/result.json', 'r') as f:
            score = json.load(f).get('final_suc')
    except:
        score = -np.inf
    return score


In [41]:
def get_scores_from_seed_sentense(seed, sentense_type, text_type='random_instruction'):

    experiment_name = f'mill_{text_type}_{seed}_1e-07'
    result_path = f'./{experiment_name}/eval/first_last_sentence_type_{sentense_type}/random_seed_*'
    result_dir = natsorted(glob.glob(result_path))
    result_files = natsorted(glob.glob(f'{result_dir[0]}/evaluated_gifs/*'))

    test_score = []
    train_score = []
    for file in result_files:
        # print(f"{file}/result.json")
        if file[-5:] == 'train':
            train_score.append(get_score_from_json(file))
        else:
            test_score.append(get_score_from_json(file))
    assert(len(train_score) == len(test_score) == 16)
    return train_score, test_score


def get_final_score(train_score, test_score):
    
    # get final score which is the test score at the highest train score.
    epochs = [i for i, x in enumerate(train_score) if x == max(train_score)]

    return np.max([test_score[i] for i in epochs])


In [43]:
sentense_type= 'image' # or image
# seed= '1' # or image
final_score_list = []
for seed in [1, 2, 3]:
    train_score, test_score = get_scores_from_seed_sentense(seed, sentense_type)
    final_score = get_final_score(train_score, test_score)
    final_score_list.append(final_score)
print(final_score_list)
print('mean', np.around(np.mean(final_score_list), 4), 'std', np.around(np.std(final_score_list, ddof=1), 4))

[0.722972972972973, 0.713963963963964, 0.6779279279279279]
mean 0.705 std 0.0238


# PETNet* (67.6 %)

In [44]:
# load from numpyz.
def get_data(file):
    d = np.load(file, allow_pickle=True)
    return pd.DataFrame(d['arr_1'], columns=d['arr_0'])

In [45]:
pd_list = []
file = './petnet_image.npz'
all_result = get_data(file)
all_result['epoch'] = [str((i+1) * 25) for i in range(16)]
all_result = all_result.set_index('epoch', drop=True).round(3)
# all_result.rename(columns=lambda x: x.replace("_15types_1_1e-07", "").replace("task_id_1", "task_id").replace("new_test", "newtest"), inplace=True)
all_result

,petnet_image_test_1,petnet_image_train_1,petnet_image_new_test_1,petnet_image_test_2,petnet_image_train_2,petnet_image_new_test_2,petnet_image_test_3,petnet_image_train_3,petnet_image_new_test_3,petnet_image_test_100,petnet_image_train_100,petnet_image_new_test_100,petnet_image_test_200,petnet_image_train_200,petnet_image_new_test_200,petnet_image_test_300,petnet_image_train_300,petnet_image_new_test_300
epoch,,,,,,,,,,,,,,,,,,
25,0.243,0.258,0.207,0.273,0.333,0.160,0.255,0.342,0.169,0.243,0.258,0.207,0.250,0.183,0.227,0.255,0.342,0.169
50,0.304,0.350,0.252,0.365,0.408,0.223,0.320,0.408,0.245,0.304,0.350,0.252,0.320,0.367,0.205,0.320,0.408,0.245
75,0.502,0.542,0.378,0.489,0.525,0.403,0.520,0.500,0.376,0.502,0.542,0.378,0.520,0.533,0.417,0.520,0.500,0.376
100,0.475,0.483,0.374,0.484,0.517,0.376,0.468,0.458,0.372,0.475,0.483,0.374,0.471,0.500,0.354,0.468,0.458,0.372
125,0.423,0.458,0.313,0.421,0.517,0.302,0.421,0.583,0.347,0.423,0.458,0.313,0.448,0.517,0.293,0.421,0.583,0.347
150,0.527,0.608,0.354,0.543,0.533,0.351,0.541,0.533,0.342,0.527,0.608,0.354,0.556,0.683,0.394,0.541,0.533,0.342
175,0.561,0.633,0.520,0.550,0.700,0.484,0.536,0.617,0.532,0.561,0.633,0.520,0.561,0.683,0.550,0.536,0.617,0.532
200,0.640,0.683,0.495,0.635,0.633,0.466,0.599,0.658,0.507,0.640,0.683,0.495,0.574,0.608,0.464,0.599,0.658,0.507
225,0.538,0.617,0.516,0.543,0.558,0.514,0.493,0.583,0.493,0.538,0.617,0.516,0.545,0.683,0.525,0.493,0.583,0.493


In [46]:
# seed= '1' # or image
final_score_list = []
seed_list = [1, 2, 3]
for seed in seed_list:
    train_score = all_result[f'petnet_image_train_{seed}'].values
    test_score = all_result[f'petnet_image_test_{seed}'].values
    final_score = get_final_score(train_score, test_score)
    final_score_list.append(final_score)
print(final_score_list)
print('mean', np.around(np.mean(final_score_list), 4), 'std', np.around(np.std(final_score_list, ddof=1), 4))

[0.691, 0.687, 0.653]
mean 0.677 std 0.0209
